# importing the MLBox

In [1]:
from mlbox.preprocessing import *
from mlbox.optimisation import *
from mlbox.prediction import *

Using Theano backend.
/home/datageek/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
paths = ["train.csv","test.csv"]
target_name = "SalePrice"

# reading and cleaning all files 

In [3]:
rd = Reader()
df = rd.run(paths, target_name)


STEP 1 - reading and building the databases ...


reading csv : train.csv ... - 0.56 s
splitting your current file into train and test sets ... - 0.02 s
reading csv : test.csv ... - 0.5 s
splitting your current file into train and test sets ... - 0.02 s
crunching your current file with the previous one ... - 0.54 s

CPU time for STEP 1: 1.76 seconds


STEP 2 - casting the databases ...


training set : casting data ... - 2.33 s
test set : casting data ... - 2.64 s

CPU time for STEP 2: 5.0 seconds


STEP 3 - analysing the training set ...


> Number of categorical features: 43
> Number of numerical features: 37
> Number of training samples : 1460
> Number of test samples : 1459

> Task : regression
> Target description :

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

more information available if calling '.get_stats(

In [4]:
cl = Cleaner()
df = cl.run(df)


STEP 4 - cleaning the databases ...


dropping duplicates on the training set ... - 0.1 s

> Number of training samples dropped : 0 out of 1460

dropping sparse features and sparse samples ... - 0.03 s

> Number of training samples dropped : 0 out of 1460
> Number of features dropped : 2
> Top 10 sparse features (%) : 
PoolQC          0.995205
MiscFeature     0.963014
Alley           0.937671
Fence           0.807534
FireplaceQu     0.472603
LotFrontage     0.177397
GarageFinish    0.055479
GarageCond      0.055479
GarageType      0.055479
GarageQual      0.055479
dtype: float64

dropping constant features ...

/home/datageek/anaconda2/lib/python2.7/site-packages/mlbox/preprocessing/cleaner.py:73: UserWarning: List is empty. No specific features will be dropped
  warnings.warn("List is empty. No specific features will be dropped")


 - 0.05 s

> Number of features dropped : 0

dropping drifting features ... - 2.2 s

> Number of features dropped : 1
> Top 10 drifting features (%) : 

Id              1.000000
2ndFlrSF        0.043727
FireplaceQu     0.042354
Exterior1st     0.040058
HeatingQC       0.037907
GrLivArea       0.034105
TotRmsAbvGrd    0.030939
BsmtFinType1    0.030811
FullBath        0.029647
MSSubClass      0.028765
dtype: float64

dropping highly-correlated features ... - 0.01 s

> Number of features dropped : 0
> Top 10 correlated features (%) : 

GarageArea    GarageCars      0.882475
GarageYrBlt   YearBuilt       0.825667
GrLivArea     TotRmsAbvGrd    0.825489
1stFlrSF      TotalBsmtSF     0.819530
2ndFlrSF      GrLivArea       0.687501
BedroomAbvGr  TotRmsAbvGrd    0.676620
BsmtFinSF1    BsmtFullBath    0.649212
GarageYrBlt   YearRemodAdd    0.642277
FullBath      GrLivArea       0.630012
2ndFlrSF      TotRmsAbvGrd    0.616423
dtype: float64



# tuning

In [5]:
mape = make_scorer(lambda y_true, y_pred: 100*np.sum(np.abs(y_true-y_pred)/y_true)/len(y_true), greater_is_better=False, needs_proba=False)
opt = Optimiser(scoring = mape, n_folds = 2)

/home/datageek/anaconda2/lib/python2.7/site-packages/mlbox/optimisation/optimiser.py:149: UserWarning: Optimiser will save all your fitted models into directory 'save/joblib'. Please clear it regularly.
  +str(self.to_path)+"/joblib'. Please clear it regularly.")


In [6]:
opt.evaluate(df)

================================================== TESTING HYPER-PARAMETERS ... ==================================================

> ENCODING: - missing values: {'numerical_strategy': 'mean', 'categorical_strategy': '<NULL>'}

            - categorical features: {'verbose': False, 'strategy': 'label_encoding'}


> MODELING: - estimation: {'num_leaves': 31, 'reg_alpha': 0, 'subsample_for_bin': 50000, 'colsample_bytree': 0.8, 'silent': True, 'max_depth': -1, 'learning_rate': 0.05, 'nthread': -1, 'min_child_weight': 5, 'strategy': 'LightGBM', 'n_estimators': 500, 'subsample': 0.9, 'reg_lambda': 0, 'subsample_freq': 1, 'max_bin': 255, 'objective': 'regression', 'min_split_gain': 0, 'seed': 0, 'min_child_samples': 10, 'boosting_type': 'gbdt'}



/home/datageek/anaconda2/lib/python2.7/site-packages/mlbox/prediction/predictor.py:62: UserWarning: No encoding strategy is specified. Default configuration is used.
  warnings.warn("No encoding strategy is specified. Default configuration is used.")
/home/datageek/anaconda2/lib/python2.7/site-packages/mlbox/prediction/predictor.py:103: UserWarning: No modeling strategy is specified. Default configuration is used.
  warnings.warn("No modeling strategy is specified. Default configuration is used.")



MEAN SCORE : custom = -16.5824926312
VARIANCE : 1.70686825527 (fold 1 = -18.2893608865, fold 2 = -14.875624376)
CPU time: 6.51516485214 seconds



-16.582492631226117

In [7]:
parameters = {
    
        "encoding" : {
        
            "missing_values" : {"numerical_strategy":{0}, "categorical_strategy":{"NULL"}},
            "categorical_features" : {"strategy":{"random_projection"}}
        
    },
    
        "modeling" : {
        
            "estimation" : {"strategy":{"LightGBM", "XGBoost"}, "colsample_bytree" : (0.6,0.9)}
        
    }
    
        }

best = opt.run(df, space=parameters, max_evals=2)


STEP 5 - tuning/evaluating the pipeline ...


================================================== TESTING HYPER-PARAMETERS ... ==================================================

> ENCODING: - missing values: {'numerical_strategy': 0, 'categorical_strategy': 'NULL'}

            - categorical features: {'verbose': False, 'strategy': 'random_projection'}


> MODELING: - estimation: {'reg_alpha': 0, 'colsample_bytree': 0.7586477678149115, 'silent': True, 'colsample_bylevel': 1.0, 'scale_pos_weight': 1, 'learning_rate': 0.05, 'missing': None, 'max_delta_step': 0, 'nthread': -1, 'base_score': 0.5, 'strategy': 'XGBoost', 'n_estimators': 500, 'subsample': 0.9, 'reg_lambda': 1, 'seed': 0, 'min_child_weight': 1, 'objective': 'reg:linear', 'max_depth': 6, 'gamma': 0}


MEAN SCORE : custom = -9.29341957883
VARIANCE : 0.828148724454 (fold 1 = -10.1215683033, fold 2 = -8.46527085437)
CPU time: 29.0859510899 seconds

================================================== TESTING HYPER-PARAMETERS ... ==

In [8]:
prd = Predictor()
prd.run(df, best)


STEP 6 - fitting the pipeline on training set ...


CPU time for STEP 6: 4.28 seconds


STEP 7 - predicting on test set ...


predicting ... - 0.12 s

> Overview on predictions : 

   SalePrice_predicted
0        122673.250000
1        154866.468750
2        187935.687500
3        195930.312500
4        185636.390625
5        174359.859375
6        167582.921875
7        170322.156250
8        179987.312500
9        128072.117188

dumping predictions into directory : save ... - 0.0 s

CPU time for STEP 7: 0.13 seconds



/home/datageek/anaconda2/lib/python2.7/site-packages/mlbox/prediction/predictor.py:366: UserWarning: Unable to get feature importances !
  warnings.warn("Unable to get feature importances !")


<mlbox.prediction.predictor.Predictor instance at 0x7f6b9e3b9518>